In [23]:
def printState(state):
    for i in range(4):
        for j in range(4):
            print("0x""%02X" % (state[j][i]), end = '  ')
        print()
        
def printRK(words):
    for i in range(0, 44):
        if i % 4 == 0 and i > 0:
            print()
        for e in words[i]:
            print("0x""%02X" % (e), end = '  ')
        print()  

In [24]:
def bytes2state(btxt):
    """ 16 바이트 숫자(바이트형 데이터)을 state(정수형 데이터)로 변환 """
    # byte => list
    txt = list(btxt)
    return [[txt[i],txt[i+1],txt[i+2],txt[i+3]] for i in range(0, 16, 4)]

In [25]:
def state2bytes(state):
    """ state(정수형 데이터)를 16 바이트 숫자(바이트형 데이터)로 변환 """
    txt = 0
    for i in range(4):
        for j in range(4):
            txt |= state[i][j]
            txt <<= 8
    txt >>= 8
    btxt = txt.to_bytes(16, byteorder='big')
    return btxt

In [26]:
Rcon128 = (
    0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36, 
)

def KeyExpansion(key):
    """ 입력 AES 16바이트 key(바이트형 데이터)에 대해 라운드키(정수형 데이터) 생성"""
    """ 0 라운드 ~ 10 라운드 44개 word """
    
    # 0 라운드키 생성 (4개 word로 구성: w[0] .. w[3])
    w = bytes2state(list(key))
    
    # w[4] .. w[43] 유도
    for i in range(4, 4 * (10 + 1)):
        # i가 4의 배수 O -> w[i] := {SubWord(RotWord(w[i-1])) xor Rcon} xor w[i-4]
        # i가 4의 배수 X -> w[i] := w[i-1] xor w[i-4]
        if i % 4 == 0:
            v = [Sbox[w[i-1][1]] ^ Rcon128[i//4 - 1] ^ w[i-4][0], \
                 Sbox[w[i-1][2]] ^ w[i-4][1], \
                 Sbox[w[i-1][3]] ^ w[i-4][2], \
                 Sbox[w[i-1][0]] ^ w[i-4][3]]
        else:
            v = [w[i-1][0] ^ w[i-4][0], \
                 w[i-1][1] ^ w[i-4][1], \
                 w[i-1][2] ^ w[i-4][2], \
                 w[i-1][3] ^ w[i-4][3]]
        w.append(v) 
    return w

In [27]:
def AddRoundKey(state, roundkey, roundnum):
    """ 입력 state와 roundkey를 xor """
    r = roundnum * 4
    for i in range(4):
        for j in range(4):
            state[i][j] ^= roundkey[r + i][j]

In [28]:
Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

Xtime2 = (
    0xc6, 0xf8, 0xee, 0xf6, 0xff, 0xd6, 0xde, 0x91, 0x60, 0x02, 0xce, 0x56, 0xe7, 0xb5, 0x4d, 0xec,
    0x8f, 0x1f, 0x89, 0xfa, 0xef, 0xb2, 0x8e, 0xfb, 0x41, 0xb3, 0x5f, 0x45, 0x23, 0x53, 0xe4, 0x9b,
    0x75, 0xe1, 0x3d, 0x4c, 0x6c, 0x7e, 0xf5, 0x83, 0x68, 0x51, 0xd1, 0xf9, 0xe2, 0xab, 0x62, 0x2a,
    0x08, 0x95, 0x46, 0x9d, 0x30, 0x37, 0x0a, 0x2f, 0x0e, 0x24, 0x1b, 0xdf, 0xcd, 0x4e, 0x7f, 0xea,
    0x12, 0x1d, 0x58, 0x34, 0x36, 0xdc, 0xb4, 0x5b, 0xa4, 0x76, 0xb7, 0x7d, 0x52, 0xdd, 0x5e, 0x13,
    0xa6, 0xb9, 0x00, 0xc1, 0x40, 0xe3, 0x79, 0xb6, 0xd4, 0x8d, 0x67, 0x72, 0x94, 0x98, 0xb0, 0x85,
    0xbb, 0xc5, 0x4f, 0xed, 0x86, 0x9a, 0x66, 0x11, 0x8a, 0xe9, 0x04, 0xfe, 0xa0, 0x78, 0x25, 0x4b,
    0xa2, 0x5d, 0x80, 0x05, 0x3f, 0x21, 0x70, 0xf1, 0x63, 0x77, 0xaf, 0x42, 0x20, 0xe5, 0xfd, 0xbf,
    0x81, 0x18, 0x26, 0xc3, 0xbe, 0x35, 0x88, 0x2e, 0x93, 0x55, 0xfc, 0x7a, 0xc8, 0xba, 0x32, 0xe6,
    0xc0, 0x19, 0x9e, 0xa3, 0x44, 0x54, 0x3b, 0x0b, 0x8c, 0xc7, 0x6b, 0x28, 0xa7, 0xbc, 0x16, 0xad,
    0xdb, 0x64, 0x74, 0x14, 0x92, 0x0c, 0x48, 0xb8, 0x9f, 0xbd, 0x43, 0xc4, 0x39, 0x31, 0xd3, 0xf2,
    0xd5, 0x8b, 0x6e, 0xda, 0x01, 0xb1, 0x9c, 0x49, 0xd8, 0xac, 0xf3, 0xcf, 0xca, 0xf4, 0x47, 0x10,
    0x6f, 0xf0, 0x4a, 0x5c, 0x38, 0x57, 0x73, 0x97, 0xcb, 0xa1, 0xe8, 0x3e, 0x96, 0x61, 0x0d, 0x0f,
    0xe0, 0x7c, 0x71, 0xcc, 0x90, 0x06, 0xf7, 0x1c, 0xc2, 0x6a, 0xae, 0x69, 0x17, 0x99, 0x3a, 0x27,
    0xd9, 0xeb, 0x2b, 0x22, 0xd2, 0xa9, 0x07, 0x33, 0x2d, 0x3c, 0x15, 0xc9, 0x87, 0xaa, 0x50, 0xa5,
    0x03, 0x59, 0x09, 0x1a, 0x65, 0xd7, 0x84, 0xd0, 0x82, 0x29, 0x5a, 0x1e, 0x7b, 0xa8, 0x6d, 0x2c
   )

Xtime3 = (
    0xa5, 0x84, 0x99, 0x8d, 0x0d, 0xbd, 0xb1, 0x54, 0x50, 0x03, 0xa9, 0x7d, 0x19, 0x62, 0xe6, 0x9a,
    0x45, 0x9d, 0x40, 0x87, 0x15, 0xeb, 0xc9, 0x0b, 0xec, 0x67, 0xfd, 0xea, 0xbf, 0xf7, 0x96, 0x5b,
    0xc2, 0x1c, 0xae, 0x6a, 0x5a, 0x41, 0x02, 0x4f, 0x5c, 0xf4, 0x34, 0x08, 0x93, 0x73, 0x53, 0x3f,
    0x0c, 0x52, 0x65, 0x5e, 0x28, 0xa1, 0x0f, 0xb5, 0x09, 0x36, 0x9b, 0x3d, 0x26, 0x69, 0xcd, 0x9f,
    0x1b, 0x9e, 0x74, 0x2e, 0x2d, 0xb2, 0xee, 0xfb, 0xf6, 0x4d, 0x61, 0xce, 0x7b, 0x3e, 0x71, 0x97,
    0xf5, 0x68, 0x00, 0x2c, 0x60, 0x1f, 0xc8, 0xed, 0xbe, 0x46, 0xd9, 0x4b, 0xde, 0xd4, 0xe8, 0x4a,
    0x6b, 0x2a, 0xe5, 0x16, 0xc5, 0xd7, 0x55, 0x94, 0xcf, 0x10, 0x06, 0x81, 0xf0, 0x44, 0xba, 0xe3,
    0xf3, 0xfe, 0xc0, 0x8a, 0xad, 0xbc, 0x48, 0x04, 0xdf, 0xc1, 0x75, 0x63, 0x30, 0x1a, 0x0e, 0x6d,
    0x4c, 0x14, 0x35, 0x2f, 0xe1, 0xa2, 0xcc, 0x39, 0x57, 0xf2, 0x82, 0x47, 0xac, 0xe7, 0x2b, 0x95,
    0xa0, 0x98, 0xd1, 0x7f, 0x66, 0x7e, 0xab, 0x83, 0xca, 0x29, 0xd3, 0x3c, 0x79, 0xe2, 0x1d, 0x76,
    0x3b, 0x56, 0x4e, 0x1e, 0xdb, 0x0a, 0x6c, 0xe4, 0x5d, 0x6e, 0xef, 0xa6, 0xa8, 0xa4, 0x37, 0x8b,
    0x32, 0x43, 0x59, 0xb7, 0x8c, 0x64, 0xd2, 0xe0, 0xb4, 0xfa, 0x07, 0x25, 0xaf, 0x8e, 0xe9, 0x18,
    0xd5, 0x88, 0x6f, 0x72, 0x24, 0xf1, 0xc7, 0x51, 0x23, 0x7c, 0x9c, 0x21, 0xdd, 0xdc, 0x86, 0x85,
    0x90, 0x42, 0xc4, 0xaa, 0xd8, 0x05, 0x01, 0x12, 0xa3, 0x5f, 0xf9, 0xd0, 0x91, 0x58, 0x27, 0xb9,
    0x38, 0x13, 0xb3, 0x33, 0xbb, 0x70, 0x89, 0xa7, 0xb6, 0x22, 0x92, 0x20, 0x49, 0xff, 0x78, 0x7a,
    0x8f, 0xf8, 0x80, 0x17, 0xda, 0x31, 0xc6, 0xb8, 0xc3, 0xb0, 0x77, 0x11, 0xcb, 0xfc, 0xd6, 0x3a
)

def SSM(state):
    # state 1행
    state[0][1],state[1][1],state[2][1], state[3][1] = state[1][1],state[2][1],state[3][1],state[0][1] 
    
    # state 2행
    state[0][2],state[1][2],state[2][2], state[3][2] = state[2][2],state[3][2],state[0][2],state[1][2] 
    
    # state 3행
    state[0][3],state[1][3],state[2][3], state[3][3] = state[3][3],state[0][3],state[1][3],state[2][3]
    
    
    
    tstate = []
    
    # 0번째 열(word)에 대한 연산
    tstate.append([ \
                   Xtime2[state[0][0]] ^ Xtime3[state[0][1]] ^ Sbox[state[0][2]] ^ Sbox[state[0][3]], \
                   Sbox[state[0][0]] ^ Xtime2[state[0][1]] ^ Xtime3[state[0][2]] ^ Sbox[state[0][3]], \
                   Sbox[state[0][0]] ^ Sbox[state[0][1]] ^ Xtime2[state[0][2]] ^ Xtime3[state[0][3]], \
                   Xtime3[state[0][0]] ^ Sbox[state[0][1]] ^ Sbox[state[0][2]] ^ Xtime2[state[0][3]]  \
                  ])
    
    # 1번째 열(word)에 대한 연산
    tstate.append([ \
                   Xtime2[state[1][0]] ^ Xtime3[state[1][1]] ^ Sbox[state[1][2]] ^ Sbox[state[1][3]], \
                   Sbox[state[1][0]] ^ Xtime2[state[1][1]] ^ Xtime3[state[1][2]] ^ Sbox[state[1][3]], \
                   Sbox[state[1][0]] ^ Sbox[state[1][1]] ^ Xtime2[state[1][2]] ^ Xtime3[state[1][3]], \
                   Xtime3[state[1][0]] ^ Sbox[state[1][1]] ^ Sbox[state[1][2]] ^ Xtime2[state[1][3]]  \
                  ])
    
    # 2번째 열(word)에 대한 연산
    tstate.append([ \
                   Xtime2[state[2][0]] ^ Xtime3[state[2][1]] ^ Sbox[state[2][2]] ^ Sbox[state[2][3]], \
                   Sbox[state[2][0]] ^ Xtime2[state[2][1]] ^ Xtime3[state[2][2]] ^ Sbox[state[2][3]], \
                   Sbox[state[2][0]] ^ Sbox[state[2][1]] ^ Xtime2[state[2][2]] ^ Xtime3[state[2][3]], \
                   Xtime3[state[2][0]] ^ Sbox[state[2][1]] ^ Sbox[state[2][2]] ^ Xtime2[state[2][3]]  \
                  ])
    
    # 3번째 열(word)에 대한 연산
    tstate.append([ \
                   Xtime2[state[3][0]] ^ Xtime3[state[3][1]] ^ Sbox[state[3][2]] ^ Sbox[state[3][3]], \
                   Sbox[state[3][0]] ^ Xtime2[state[3][1]] ^ Xtime3[state[3][2]] ^ Sbox[state[3][3]], \
                   Sbox[state[3][0]] ^ Sbox[state[3][1]] ^ Xtime2[state[3][2]] ^ Xtime3[state[3][3]], \
                   Xtime3[state[3][0]] ^ Sbox[state[3][1]] ^ Sbox[state[3][2]] ^ Xtime2[state[3][3]]  \
                  ]) 
    #print(tstate)
    #print()
    
    # state를 tstate로 대체
    for i in range(4):
        for j in range(4):
            state[i][j] = tstate[i][j]
    

In [29]:
def SubBytes(state):
    """ 입력 state에 대해 SubBytes 수행 (table look-up) """
    for i in range(4):
        for j in range(4):
            state[i][j] = Sbox[state[i][j]]

def ShiftRows(state):
    """ 입력 state에 대해 ShiftRows 수행 """
    # state 1행
    state[0][1],state[1][1],state[2][1], state[3][1] = state[1][1],state[2][1],state[3][1],state[0][1] 
    
    # state 2행
    state[0][2],state[1][2],state[2][2], state[3][2] = state[2][2],state[3][2],state[0][2],state[1][2] 
    
    # state 3행
    state[0][3],state[1][3],state[2][3], state[3][3] = state[3][3],state[0][3],state[1][3],state[2][3] 

In [30]:
def AES128EncBlock(aes128key,plaintext):
    # plaintext (바이트형 데이터) => state (정수형 데이터)
    state = bytes2state(plaintext)
    
    # Roundkey 생성
    roundkey = KeyExpansion(aes128key)
    #printRK(roundkey)
    #print()
    
    # Round 0 
    AddRoundKey(state, roundkey, 0)
    #printState(state)
    #print()
    
    #Round 1 ~ Round 9
    for i in range(1,10):
        SSM(state)
        AddRoundKey(state, roundkey, i)
        #printState(state)
        #print()
        
    # Round 10
    SubBytes(state)
    ShiftRows(state)
    AddRoundKey(state, roundkey, 10)
    #printState(state)
    #print()
    
    # state => 바이트열 반환
    cihpertext = state2bytes(state)
    return cihpertext

In [31]:
""" Test """
plaintext = (0x3243f6a8885a308d313198a2e0370734).to_bytes(16, byteorder='big')
aes128key = (0x2b7e151628aed2a6abf7158809cf4f3c).to_bytes(16, byteorder='big')
ciphertext = AES128EncBlock(aes128key,plaintext)
ciphertextNum = int.from_bytes(ciphertext,byteorder='big')
print("0x""%032X" % (ciphertextNum))

0x3925841D02DC09FBDC118597196A0B32


In [32]:
""" Test """
import time
start = time.time()
plaintext = (0x3243f6a8885a308d313198a2e0370734).to_bytes(16, byteorder='big')
aes128key = (0x2b7e151628aed2a6abf7158809cf4f3c).to_bytes(16, byteorder='big')
for i in range(10000):
    AES128EncBlock(aes128key,plaintext)
print("time (sec) :", time.time() - start)  

time (sec) : 2.7556331157684326
